In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Importing nltk library for Sentiment score and POS tagging 
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus.reader import TaggedCorpusReader 
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from nltk.corpus import stopwords

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

import string
import statsmodels.api as sm







[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [0]:

#using dataset that contains all features till image features extracted.
dataframe = pd.read_excel('/content/drive/My Drive/DATASET FINAL/DATASET.xlsx')

print(dataframe.columns)
happy = dataframe[dataframe.label == 1]
angry = dataframe[dataframe.label == 2]
depressed = dataframe[dataframe.label == 3]
fear = dataframe[dataframe.label == 4]

#choosing 2100 post for each label
happy = happy.iloc[0:2100, ]
angry = angry.iloc[0:2100, ]
depressed = depressed.iloc[0:2100, ]
fear = fear.iloc[0:2100, ]

frames = [ happy, angry,depressed, fear]

dataframe = pd.concat(frames)
#shuffling
dataframe = shuffle(dataframe)
dataframe.reset_index(inplace=True, drop=True)
print(dataframe.head(10))


Index(['alt', 'en', 'firstComment', 'imageUrl', 'likesCount', 'locationName',
       'ownerUsername', 'timestamp', 'url', 'faceCount', 'H', 'S', 'V',
       'label', 'followersCount', 'followsCount', 'bio', 'postsCount', 'lc/fc',
       'fc/foc', 'animal', 'in-outdoor', 'text', 'people', 'other'],
      dtype='object')
                                                 alt  en  ... people other
0                    No photo description available.  en  ...      0   NaN
1    Image may contain: coffee, coffee cup and drink  en  ...      0   NaN
2                        Image may contain: 1 person  en  ...      1   NaN
3  Image may contain: one or more people and peop...  en  ...      1   NaN
4  Image may contain: sky, tree, cloud, outdoor a...  en  ...      0   NaN
5  Image may contain: one or more people, people ...  en  ...      1   NaN
6                    No photo description available.  en  ...      0   NaN
7               Image may contain: 1 person, closeup  en  ...      1   NaN
8   

In [0]:
# dropping column
col = ['locationName', 'ownerUsername', 'alt', 'en',  'imageUrl', 'timestamp', 'url', "bio"]

dataframe = dataframe.drop(col, axis=1)

cols = [ 'label','firstComment', 'textWeight', 'H', 'S','V', 'followersCount', 'fc/foc', 'likesCount', 'faceCount',  'followsCount', 'postsCount', 'lc/fc', 'in-outdoor']
#Index(['alt', 'en', 'firstComment', 'imageUrl', 'likesCount', 'locationName',
#       'ownerUsername', 'timestamp', 'url', 'faceCount', 'H', 'S', 'V',
#       'label', 'followersCount', 'followsCount', 'bio', 'postsCount', 'lc/fc',
#       'fc/foc', 'animal', 'in-outdoor', 'text', 'people', 'other'],

dataframe = dataframe.reindex(columns = cols)

print(dataframe)
print(dataframe.columns)


      label  ... in-outdoor
0         3  ...          0
1         4  ...          0
2         2  ...          0
3         1  ...          0
4         4  ...          2
...     ...  ...        ...
8395      3  ...          0
8396      3  ...          2
8397      1  ...          0
8398      2  ...          0
8399      4  ...          0

[8400 rows x 14 columns]
Index(['label', 'firstComment', 'textWeight', 'H', 'S', 'V', 'followersCount',
       'fc/foc', 'likesCount', 'faceCount', 'followsCount', 'postsCount',
       'lc/fc', 'in-outdoor'],
      dtype='object')


TF-IDF weights being extracted. 

In [0]:
v = TfidfVectorizer(analyzer=lambda d: d.split(', ')).fit(dataframe['firstComment'])
tfidf = v.fit_transform(dataframe['firstComment'])
print(v.get_feature_names())
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(tfidf)
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': v.get_feature_names(), 'weight': weights})

dataframe['textWeight'] = weights_df.weight
print(dataframe.head(5))
#.sort_values(by='weight', ascending=True)
print(weights_df.head(50))

['', "\nBut this insecurities will creep in. \nI'm really sorry. \nI've seen her", '\nHow could you downgrade a signature of Kokotiam??\nEven if you put the price of Starbucks', "\nMemories are made. \nMemories are worth more than gold.\n©Sunshine\nIt's a new week", '\nNAKED AND NOT ASHAMED.\n\nA must read for everyone! N2500 for a copy.\n\nThe first book reading was done by Mrs Olaitan  Ajayi', '\nWhich I definitely needed.\nWith how I was feeling,\nI needed to gather some courage and motivation to do it.\nBefore I got there,\nI applied Valor on my spine', '\nand the most delicious bruschetta served with salmon', '\ncompared with 85 out of 148 in the placebo group\n\n#pregnant \n#pregnancy \n#miscarriage \n#threatenedmiscarriage \n#recurrentmiscarriage \n#gynaecologist_london', "\nfear of change,\n\nHow many times do you use procrastination as a reason not to start something you have dreamed about?\n\nHow many times have you procrastnated about starting something because you didn't fe

Cleaning text data


In [0]:
dataframe["text_lower"] = dataframe["firstComment"].str.lower()
dataframe.head()
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

dataframe["text_wo_punct"] = dataframe["text_lower"].apply(lambda text: remove_punctuation(text))
dataframe.head()

nltk.download('stopwords')
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

dataframe["text_wo_stop"] = dataframe["text_wo_punct"].apply(lambda text: remove_stopwords(text))
dataframe.head()
import re
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
dataframe["text_wo_emoji"]=dataframe["text_wo_stop"].apply(lambda text: remove_emoji(text))
dataframe.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,label,firstComment,textWeight,H,S,V,followersCount,fc/foc,likesCount,faceCount,followsCount,postsCount,lc/fc,in-outdoor,text_lower,text_wo_punct,text_wo_stop,text_wo_emoji
0,3,Moloken • Our Astral Circle,0.000144,67.705093,0.180147,39.026358,831.0,2.415698,34,0,344.0,400.0,0.040915,0,moloken • our astral circle,moloken • our astral circle,moloken • astral circle,moloken • astral circle
1,4,Coffee time ☕️\n#coffee #caffeine #cappucino #...,0.000042,102.291824,0.538909,34.010189,156.0,0.968944,12,0,161.0,110.0,0.076923,0,coffee time ☕️\n#coffee #caffeine #cappucino #...,coffee time ☕️\ncoffee caffeine cappucino angr...,coffee time ☕️ coffee caffeine cappucino angry...,coffee time coffee caffeine cappucino angrypi...
2,2,When people make up stories about you.. #irrit...,0.000042,81.760147,0.406251,41.695380,202.0,1.181287,15,0,171.0,89.0,0.074257,0,when people make up stories about you.. #irrit...,when people make up stories about you irritate...,people make stories irritatedlook,people make stories irritatedlook
3,1,Been dine like 4 flat tires /@_jenkxxs_ !,0.000060,74.659760,0.205714,51.527369,6596.0,2.896794,240,1,2277.0,683.0,0.036386,0,been dine like 4 flat tires /@_jenkxxs_ !,been dine like 4 flat tires jenkxxs,dine like 4 flat tires jenkxxs,dine like 4 flat tires jenkxxs
4,4,"At centre parks, going up there in a minute! O...",0.000049,183.501938,0.301659,50.345890,109.0,0.668712,7,0,163.0,199.0,0.064220,2,"at centre parks, going up there in a minute! o...",at centre parks going up there in a minute omg...,centre parks going minute omg midlife crisis a...,centre parks going minute omg midlife crisis a...


POS tagging


In [0]:
for index,row in dataframe.iterrows():
      parag=dataframe.loc[index,'text_wo_emoji']
      para=nltk.tokenize.word_tokenize(parag)
      tagged = nltk.pos_tag(para)
      adverb=0
      adjective=0
      verb=0
      noun=0
      mylist=[]
      for i in range(len(tagged)):
        if tagged[i][1]=='NN' or  tagged[i][1]=='NNS' or tagged[i][1]=='NNP' or tagged[i][1]=='NNPS':
          noun=noun+1
        elif tagged[i][1]=='VB' or tagged[i][1]=='VBD' or tagged[i][1]=='VBN' or tagged[i][1]=='VBP' or tagged[i][1]=='VBZ' or tagged[i][1]=='VBG':
          verb=verb+1
        elif tagged[i][1]=='JJ' or tagged[i][1]=='JJR' or tagged[i][1]=='JJS':
          adjective=adjective+1
        elif tagged[i][1]=='RB' or tagged[i][1]=='RBR' or tagged[i][1]=='RBS':
          adverb=adverb+1
        else:
          pass
      dataframe.loc[index,'noun']=noun
      dataframe.loc[index,'verb']=verb
      dataframe.loc[index,'adjective']=adjective
      dataframe.loc[index,'adverb']=adverb
   
dataframe.head()

,label,firstComment,textWeight,H,S,V,followersCount,fc/foc,likesCount,faceCount,followsCount,postsCount,lc/fc,in-outdoor,text_lower,text_wo_punct,text_wo_stop,text_wo_emoji,noun,verb,adjective,adverb
0,3,Moloken • Our Astral Circle,0.000144,67.705093,0.180147,39.026358,831.0,2.415698,34,0,344.0,400.0,0.040915,0,moloken • our astral circle,moloken • our astral circle,moloken • astral circle,moloken • astral circle,1.0,1.0,2.0,0.0
1,4,Coffee time ☕️\n#coffee #caffeine #cappucino #...,0.000042,102.291824,0.538909,34.010189,156.0,0.968944,12,0,161.0,110.0,0.076923,0,coffee time ☕️\n#coffee #caffeine #cappucino #...,coffee time ☕️\ncoffee caffeine cappucino angr...,coffee time ☕️ coffee caffeine cappucino angry...,coffee time coffee caffeine cappucino angrypi...,8.0,0.0,0.0,0.0
2,2,When people make up stories about you.. #irrit...,0.000042,81.760147,0.406251,41.695380,202.0,1.181287,15,0,171.0,89.0,0.074257,0,when people make up stories about you.. #irrit...,when people make up stories about you irritate...,people make stories irritatedlook,people make stories irritatedlook,2.0,2.0,0.0,0.0
3,1,Been dine like 4 flat tires /@_jenkxxs_ !,0.000060,74.659760,0.205714,51.527369,6596.0,2.896794,240,1,2277.0,683.0,0.036386,0,been dine like 4 flat tires /@_jenkxxs_ !,been dine like 4 flat tires jenkxxs,dine like 4 flat tires jenkxxs,dine like 4 flat tires jenkxxs,2.0,1.0,1.0,0.0
4,4,"At centre parks, going up there in a minute! O...",0.000049,183.501938,0.301659,50.345890,109.0,0.668712,7,0,163.0,199.0,0.064220,2,"at centre parks, going up there in a minute! o...",at centre parks going up there in a minute omg...,centre parks going minute omg midlife crisis a...,centre parks going minute omg midlife crisis a...,6.0,2.0,1.0,0.0


Sentiment analyzer

In [0]:
sid = SentimentIntensityAnalyzer()
for index,row in dataframe.iterrows():
  parag=dataframe.loc[index,'text_wo_emoji']
  scores = sid.polarity_scores(parag)
  dataframe.loc[index,'compound']=scores['compound']
  dataframe.loc[index,'neg']=scores['neg']
  dataframe.loc[index,'neu']=scores['neu']
  dataframe.loc[index,'pos']=scores['pos']
print(dataframe.head())

   label                                       firstComment  ...    neu    pos
0      3                        Moloken • Our Astral Circle  ...  1.000  0.000
1      4  Coffee time ☕️\n#coffee #caffeine #cappucino #...  ...  1.000  0.000
2      2  When people make up stories about you.. #irrit...  ...  1.000  0.000
3      1          Been dine like 4 flat tires /@_jenkxxs_ !  ...  0.615  0.385
4      4  At centre parks, going up there in a minute! O...  ...  0.661  0.000

[5 rows x 26 columns]


In [0]:
dataframe = dataframe.drop([ 'text_lower', 'text_wo_punct', 'text_wo_stop'], axis=1)

dataframe.head()

,label,firstComment,textWeight,H,S,V,followersCount,fc/foc,likesCount,faceCount,followsCount,postsCount,lc/fc,in-outdoor,text_wo_emoji,noun,verb,adjective,adverb,compound,neg,neu,pos
0,3,Moloken • Our Astral Circle,0.000144,67.705093,0.180147,39.026358,831.0,2.415698,34,0,344.0,400.0,0.040915,0,moloken • astral circle,1.0,1.0,2.0,0.0,0.0000,0.000,1.000,0.000
1,4,Coffee time ☕️\n#coffee #caffeine #cappucino #...,0.000042,102.291824,0.538909,34.010189,156.0,0.968944,12,0,161.0,110.0,0.076923,0,coffee time coffee caffeine cappucino angrypi...,8.0,0.0,0.0,0.0,0.0000,0.000,1.000,0.000
2,2,When people make up stories about you.. #irrit...,0.000042,81.760147,0.406251,41.695380,202.0,1.181287,15,0,171.0,89.0,0.074257,0,people make stories irritatedlook,2.0,2.0,0.0,0.0,0.0000,0.000,1.000,0.000
3,1,Been dine like 4 flat tires /@_jenkxxs_ !,0.000060,74.659760,0.205714,51.527369,6596.0,2.896794,240,1,2277.0,683.0,0.036386,0,dine like 4 flat tires jenkxxs,2.0,1.0,1.0,0.0,0.3612,0.000,0.615,0.385
4,4,"At centre parks, going up there in a minute! O...",0.000049,183.501938,0.301659,50.345890,109.0,0.668712,7,0,163.0,199.0,0.064220,2,centre parks going minute omg midlife crisis a...,6.0,2.0,1.0,0.0,-0.6249,0.339,0.661,0.000


Saving to Drive

In [0]:
dataframe.to_excel('/content/drive/My Drive/DATASET FINAL/DatasetFinal.xlsx', sheet_name='sheet1', index = False)